<a href="https://colab.research.google.com/github/harperd/machine-learning/blob/master/notebooks/multiclass-logistic-regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Logistic Regression

Use logistic regression to recognize hand-written digits (0 to 9).

## Imports

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import google.colab as colab
import scipy.optimize as opt
import io

from scipy.io import loadmat
from scipy.optimize import minimize

# Allow saving our graphs in the notebook
%matplotlib inline

style.use('dark_background')

## Read Sample Data

In [3]:
mat_file = colab.files.upload()
!ls -l

Saving ex3data1.mat to ex3data1.mat
total 7340
-rw-r--r-- 1 root root 7511764 Aug 10 17:54 ex3data1.mat
drwxr-xr-x 1 root root    4096 Aug  2 16:06 sample_data


In [4]:
mat_data = loadmat('ex3data1.mat')
mat_data

{'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [5]:
X = mat_data['X']
# Add bias column
X = np.hstack((np.ones(X.shape[0])[:, np.newaxis], X))

y = mat_data['y']
theta = np.array(np.zeros(X.shape[1]), ndmin = 2)

print(f'X Shape: {X.shape}')
print(f'y Shape: {y.shape}')
print(f'Theta Shape: {theta.shape}')

X Shape: (5000, 401)
y Shape: (5000, 1)
Theta Shape: (1, 401)


![Hand written numbers](https://github.com/harperd/machine-learning/blob/master/images/ex3-1.png?raw=1)

In [0]:
def sigmoid(z):
  return 1 / ( 1 + np.exp(-z) )

In [0]:
def compute_hypothesis(theta, X):
  # Compute our hypothesis
  z = X @ theta.T
  
  # Scale our hypothesis using Sigmoid.
  # Here, if the parameter is zero then the sigmoid value will be 0.5.
  h = sigmoid(z)
  
  return h

In [0]:
def compute_cost(theta, X, y, alpha):
  # Fix for minimize function
  theta = np.array(theta, ndmin = 2)
  X = np.array(X, ndmin = 2)
  y = np.array(y, ndmin = 2)
  
  # Compute our hypothesis
  h = compute_hypothesis(theta, X)
  
  first = np.log(h) * -y
  second = np.log(1 - h) * (1 - y)
  
  # The number of examples
  m = len(y)
  
  # Implement Ridge Regression (L2 Regularization)
  #
  # Get all theta values except theta0 out
  # intercept term
  num_params = theta.shape[1]
  params = theta[:, 1:num_params]
  
  # Set our lambda value
  lamb = alpha / ( m * 2 )
  
  # Compute our regularization term for cost
  reg = lamb * np.sum(params ** 2)
  
  # Compute our cost with regularization
  cost = ( np.sum(first - second) / m ) + reg
  
  return cost

In [0]:
def compute_gradient(theta, X, y, alpha):
  # Fix for minimize function
  theta = np.array(theta, ndmin = 2)
  X = np.array(X, ndmin = 2)
  y = np.array(y, ndmin = 2)
  
  # Compute our hypothesis
  h = compute_hypothesis(theta, X)

  # Get the error
  error = h - y

  # The number of examples
  m = len(y)
  
  # Set our lambda value
  lamb = alpha / ( m * 2 )
  
  # Compute our regularization term for gradient descent
  reg = 1 - (alpha * (lamb / m))
  
  # Calculate the gradient
  gradient = ( theta * reg ) - ( alpha * ((error.T @ X) / m) )
  
  return gradient

In [0]:
def cgrad(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()
  
def ccost(theta, X, y, learningRate):
  theta = np.matrix(theta)
  X = np.matrix(X)
  y = np.matrix(y)
  first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
  second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
  reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
  return np.sum(first - second) / (len(X)) + reg

In [10]:
def train_model(theta, X, y, alpha):
  num_classes = len(np.unique(y))
  num_params = theta.shape[1]
  theta_min = np.ones((num_classes, num_params))
  
  for k in range(1, num_classes + 1):
    print(f'Optimizing theta values for class {k}... ', end = '')

    y_train = [ 1 if K[0] == k else 0 for K in y ]
    y_train = np.array(y_train, ndmin = 2).T

    result = minimize(
        method = 'TNC',
        fun = compute_cost, jac = compute_gradient,
        #fun = ccost, jac = cgrad,
        x0 = theta,
        args = ( X, y_train, alpha ))

    #result = opt.fmin_tnc(
    #      # Initial guess.
    #      x0 = theta,
    #      # Objective function to be minimized.
    #      func = compute_cost,
    #      # Gradient of func.
    #      fprime = compute_gradient,
    #      # Extra arguments passed to f and fprime.
    #      args = ( X, y_train, alpha ))

    theta_min[k - 1] = result.x #result[0]
    
    cost = compute_cost(result.x, X, y_train, alpha)
    
    if(result.message):
      print(result.message)
    else:
      print(f'Iterations = {result.nit}, cost = {cost}')
  
  return theta_min

theta_min = train_model(theta, X, y, 1)

Optimizing theta values for class 1... Linear search failed
Optimizing theta values for class 2... Linear search failed
Optimizing theta values for class 3... Linear search failed
Optimizing theta values for class 4... Linear search failed
Optimizing theta values for class 5... Linear search failed
Optimizing theta values for class 6... Linear search failed
Optimizing theta values for class 7... Linear search failed
Optimizing theta values for class 8... Linear search failed
Optimizing theta values for class 9... Linear search failed
Optimizing theta values for class 10... Linear search failed


In [0]:
def make_predictions(theta, X):
  # Compute our hypothesis
  h = compute_hypothesis(theta, X)
  
  # Get the index of each max probability for each
  # of the 5k examples where index number is the 
  # classifier.
  h = ( np.argmax(h, axis = 1) ) + 1
  
  return h

def compute_accuracy(predictions, y):
  # Get the correct predictions where correct is 1 and
  # incorrect is 0.
  correct = [ 
      1 if p_val == y_val else 0 
      # The purpose of zip() is to map the similar index of multiple 
      # containers so that they can be used just using as single entity.
      for (p_val, y_val) in zip(predictions, y)
  ]
  
  # Calculate the overall accuracy.
  accuracy = sum(correct) / len(correct)
  
  return accuracy

h = make_predictions(theta_min, X)
accuracy = compute_accuracy(h, y)

print(f'Model accuracy: {accuracy * 100}%')

Model accuracy: 10.0%
